# Intelligent system for monitoring indoor air quality and fight against COVID-19.<br>
**Dataframe to .csv file**.<br>
Notebook developed in Python with the aim of query data stored in a NoSQL database hosted in Azure called COSMOSDB in order to save it in a .csv file.<br>This allows to work with data science tools and also user friendly tools like Excel.<br>
Author: Felix Angel Martinez Muela

<img src="https://licensebuttons.net/l/by-nc-sa/3.0/88x31.png"><br>
**Attribution-NonCommercial-ShareAlike**<br>
**CC BY-NC-SA**

This license lets others remix, adapt, and build upon your work non-commercially, as long as they credit you and license their new creations under the identical terms.<br>
[View License Deed](https://creativecommons.org/licenses/by-nc-sa/4.0) | [View Legal Code](https://creativecommons.org/licenses/by-nc-sa/4.0/legalcode)

Installation of necessary libraries.

In [1]:
%%capture
!pip install pandas
!pip install azure-cosmos

In [2]:
import os
import time
import datetime
import json
from azure.cosmos import CosmosClient
import pandas as pd

Starting measuring execution time.

In [3]:
start_time = time.time()

Environment variables.

In [4]:
os.environ['ACCOUNT_URI'] = "https://airq-cosmos.documents.azure.com:443/"
os.environ['ACCOUNT_KEY'] = "s8bh08THd91Yt6nNbZDN99r6fHH212UCr8l4cuP7v7ySUj3Nps5DDUgIgmIix4QnP481e99UK6NbaQ99AfO9dg=="

CosmosDB client.

In [5]:
url = os.environ['ACCOUNT_URI']
key = os.environ['ACCOUNT_KEY']

client = CosmosClient(url, credential=key)
database_name = 'airq-db'
database = client.get_database_client(database_name)
container_name = 'telemetry'
container = database.get_container_client(container_name)

NoSQL Data base (CosmosDB) properties.

In [6]:
properties = database.read()
print(json.dumps(properties))

{"id": "airq-db", "_rid": "vohrAA==", "_self": "dbs/vohrAA==/", "_etag": "\"0000689f-0000-0d00-0000-60b5ee1b0000\"", "_colls": "colls/", "_users": "users/", "_ts": 1622535707}


Unix Timestamp a week before.

In [7]:
last_x_days=7

In [8]:
yesterday = datetime.date.today() - datetime.timedelta(last_x_days)
unix_time= yesterday.strftime("%s") #Second as a decimal number [00,61] (or Unix Timestamp)
print(unix_time)

1623801600


Query only  data of the following device.

In [9]:
deviceid='"24:A1:60:58:09:94"'

Database query.

In [10]:
query= 'SELECT * FROM c WHERE (c.deviceid = {deviceid}) AND (c.time>{unix_time}) ORDER BY c._ts ASC'.format(deviceid=deviceid, unix_time=unix_time)

Query to the database and document concatenation to a list.

In [11]:
%%capture
dflist = []
for item in container.query_items(
        query=query,
        enable_cross_partition_query=True):
    print(json.dumps(item, indent=True))
    dflist.append(dict(item))

Explanation of the data received by the CosmosDB database.<br>
Data: <br>
0.   **deviceid**: MAC direction of the device's wifi.
1.   **time**: time at which the measured values were collected with UTC time. Value generated per IoT device.
2.   **temperature**: temperature in Celsius degrees. Value generated by BME280 sensor.
3.   **humidity**: relative humidity percentage %. Value generated by BME280 sensor.
4.   **pressure**: atmospheric pressure in hPa. Value generated by BME280 sensor.
5.   **altitude**: altitude at which the device is located as a function of its atmospheric pressure. Indicative value and that takes as reference the atmospheric pressure at sea level: 1013.25 hPa. Value generated by BME280 sensor.
6.   **co2**: CO2 particles measured in ppm. Value generated by MHZ19 sensor.
7.   **rawco2**: value captured by the sensor regardless of calibration. Value generated by MHZ19 sensor.
8.   **pm25**: 2.5 micron airborne particles measured in ppm. Value generated by SDS011 sensor.
9.   **pm10** 10 micron airborne particles measured in ppm. Value generated by SDS011 sensor.
10.   **tvoc**: Volatile Organic Compound particles in suspension in the air, measured in ppb. Value generated by sensor.
11.   **eco2**: CO2 particles measured in ppm. Value generated by SGP30 sensor.
12.   **rawh2**: dihydrogen particles in the air. Value needed to measure eCO2 and TVOC. Value generated by SGP30 sensor.
13.   **rawethanol**: ethanol particles in the air. Value required to measure eCO2 and TVOC. Value generated by SGP30 sensor.
14.   **lux**: value indicating the brightness found in the environment, measured in lx, equivalent to 1 lm/m² Value generated by VEML7700 sensor.
15.   **whitelux**: value indicating the brightness of the white color found in the environment, measured in lx, equivalent to 1 lm/m². Value generated by VEML7700 sensor.
16.   **rawals**: raw value to obtain lux and whitelux. Value generated by VEML7700 sensor.
17.   **id**: indicates the document identifier. Value generated by the database.
18.   **_rid**: indicates the document identifier. Value generated by the database.
19.   **_self**: indicates the document URI. Value generated by the database.
20.   **_etag**: value necessary to manage the simultaneity of the documents. Value generated by the database.
21.   **_attachments**: indicates attachments associated with the document. Value generated by the database.
22.   **_ts**: indicates when the document was last modified. Value generated by the database.

Creation of the dataframe from the query.

In [12]:
df = pd.DataFrame(dflist)

We list the number of devices that have sent data.

In [13]:
devices = df.deviceid.unique()
print(devices)

['24:A1:60:58:09:94']


Select from the data frame only the rows that belong to the device you are interested in.

In [14]:
df = df.loc[df['deviceid'] == '24:A1:60:58:09:94']

We eliminate from the DF the data that do not add value.<br>
The values that for the development of the model do not contribute value are the following: deviceid, altitude, rawco2, rawh2, rawethanol, rawals, id, _rid, _self, _etag, attachements, _ts.

In [15]:
df = df.drop(['deviceid','altitude','rawco2','rawh2','rawethanol','rawals','id','_rid','_self','_etag','_attachments','_ts'], 1)

In [16]:
df = df.rename(columns={'time': 'Date'}) # Adjustment of column names
df['Date']= pd.to_datetime(df['Date'], unit='s') # Conversion to date type
df = df.sort_values('Date') # Sort by date
df['Date'] = df['Date'].dt.floor('min') # Approaching seconds to minutes
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9192 entries, 0 to 9191
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         9192 non-null   datetime64[ns]
 1   temperature  9192 non-null   float64       
 2   humidity     9192 non-null   float64       
 3   pressure     9192 non-null   float64       
 4   co2          9192 non-null   int64         
 5   pm25         9192 non-null   float64       
 6   pm10         9192 non-null   float64       
 7   tvoc         9192 non-null   int64         
 8   eco2         9192 non-null   int64         
 9   lux          9192 non-null   float64       
 10  whitelux     9192 non-null   float64       
dtypes: datetime64[ns](1), float64(7), int64(3)
memory usage: 861.8 KB


In [17]:
df.head()

,Date,temperature,humidity,pressure,co2,pm25,pm10,tvoc,eco2,lux,whitelux
0,2021-06-16 00:00:00,27.45,40.90,933.26,697,3.4,6.1,92,2114,0.0,0.0
1,2021-06-16 00:02:00,27.47,40.78,933.23,684,3.1,6.4,89,2087,0.0,0.0
2,2021-06-16 00:03:00,27.47,40.76,933.24,686,3.1,5.1,83,2061,0.0,0.0
3,2021-06-16 00:04:00,27.50,40.53,933.24,690,3.2,5.1,84,2062,0.0,0.0
4,2021-06-16 00:05:00,27.52,40.46,933.24,697,3.4,5.8,77,2096,0.0,0.0




---

**Dataframe to .csv file**.

Directory where the dataframe is going to be stored.

In [18]:
%%capture
!mkdir data/

Process of importing dataframe to the .csv file.

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html


In [19]:
df.to_csv('data/airq_data.csv', float_format='%.3f', decimal=",", sep = ';', encoding='ISO-8859-1', index=False)

Execution time finished.

In [20]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 18.16928768157959 seconds ---
